# Isopsephical riddle of the Pseudo Sibylline hexameter

Sibyl Book 1 lines 137 - 146:


<blockquote><pre>
Sibyl. 1:137 εἰμὶ δ᾿ ἔγωγε ὁ ὤν, σὺ δ᾿ ἐνί φρεσὶ σῇσι νόησον·
Sibyl. 1:138 οὐρανὸν ἐνδέδυμαι, περιβέβλημαι δὲ θάλασσαν,
Sibyl. 1:139 γαῖα δέ μοι στήριγμα ποδῶν, περὶ σῶμα κέχυται
Sibyl. 1:140 ἀὴρ ἠδ᾿ ἄστρων με χορὸς περιδέδρομε πάντη.
Sibyl. 1:141 ἐννέα γράμματ᾿ ἔχω· τετρασύλλαβός εἰμι· νόει με·
Sibyl. 1:142 αἱ τρεῖς αἱ πρῶται δύο γράμματ᾿ ἔχουσιν ἑκάστη,
Sibyl. 1:143 ἡ λοιπὴ δὲ τὰ λοιπὰ καί εἰσιν ἄφωνα τὰ πέντε·
Sibyl. 1:144 τοῦ παντὸς δ᾿ ἀριθμοῦ ἑκατοντάδες εἰσὶ δὶς ὀκτώ
Sibyl. 1:145 καὶ τρεῖς, τρὶς δεκάδες σύν γ᾿ ἑπτά. γνοὺς δὲ τίς εἰμι
Sibyl. 1:146 οὐκ ἀμύητος ἔσῃ τῆς παρ᾿ ἐμοὶ σοφίης.</pre>
</blockquote>

Translation:

<blockquote>
And I am He who is, and in thy heart
Do thou discern. I clothe me with the heaven,
And cast the sea around me, and for me
Earth is a footstool, and the air is poured
Around my body; and on every side
Around me runs the chorus of the stars.
Nine letters have I; of four syllables
I am; discern me. The first three have each
Two letters, the remaining one the rest,
And five are mates; and of the entire sum
The hundreds are twice eight and thrice three tens
Along with seven. Now, knowing who I am,
Be thou not uninitiate in my lore.
</blockquote>


## Process Greek Corpora

First task is to get a big raw text to operate with. CLTK provides an importer to the Perseus open source data source. Other good option would be to use TLG: http://opengreekandlatin.github.io/First1KGreek/

In [1]:
import sys
# uncomment to install cltk
#!{sys.executable} -m pip install cltk
sys.version

'3.6.1 |Anaconda 4.4.0 (64-bit)| (default, May 11 2017, 13:25:24) [MSC v.1900 64 bit (AMD64)]'

In [2]:
# uncomment to install abnum
#!{sys.executable} -m pip install abnum

In [3]:
from cltk.corpus.utils.importer import CorpusImporter
corpus_importer = CorpusImporter('greek')
', '.join(corpus_importer.list_corpora)

'greek_software_tlgu, greek_text_perseus, phi7, tlg, greek_proper_names_cltk, greek_models_cltk, greek_treebank_perseus, greek_lexica_perseus, greek_training_set_sentence_cltk, greek_word2vec_cltk, greek_text_lacus_curtius, greek_text_first1kgreek'

In [4]:
import shutil, errno
from os import path, listdir, remove, makedirs

# file path utility for win/mac/lin
def joinpaths(d, paths):
    for p in paths:
        d = path.join(d, p)
    return d

# append (a) or rewrite (w) a file
def append_to_file(file, string, mode = 'a'):
    with open(file, mode, encoding="utf-8", newline='\n') as f:
        f.write(string)

# copy files utility
def copy(src, dest):
    try:
        ignore = shutil.ignore_patterns('*.csv*', '*lat.xml', '*lat1.xml', '*cop1.xml', '*eng1.xml', '__cts__.xml')
        shutil.copytree(src, dest, ignore=ignore)
    except OSError as e:
        # error was caused because the source wasn't a directory
        if e.errno == errno.ENOTDIR:
            shutil.copy(src, dest)
        else:
            print('Directory not copied. Error: %s' % e)

# home dir
home = path.expanduser("~")

# filter dirs and files to get all greek authors
# directory: by default corpora is downloaded to the user root under cltk_data
dirt = joinpaths(home, ["cltk_data", "greek", "text"])

# init corpora
corporas = ["greek_text_perseus", "greek_text_tlg"]

# import corpora
for corpus in ["greek_text_perseus", "greek_text_first1kgreek"]:
    try:
        corpus_importer.import_corpus(corpus)
    except Exception as e:
        print(e)

# copy all suitable greek text files from the source dir to the destination work dir
if not path.isdir(path.join(dirt, "greek_text_tlg")):
    src = joinpaths(dirt, ["greek_text_first1kgreek", "data"])
    dst = joinpaths(dirt, ["greek_text_tlg"])
    print("Copying %s -> %s" % (src, dst))
    try:
        copy(src, dst)
    except Exception as e:
        print(e)

In [5]:
# download greek betacode decoder and import
file_bc = 'betacode.py'
if not path.isfile(file_bc):
    try:
        # python 3.0 and later
        from urllib.request import urlopen
    except ImportError:
        # python 2
        from urllib2 import urlopen
    response = urlopen('https://raw.githubusercontent.com/markomanninen/grcriddles/master/betacode.py')
    append_to_file(file_bc, response.read().decode("utf-8"), 'w')

from betacode import betacode_to_unicode

In [7]:
import re
from bs4 import BeautifulSoup
from xml.dom import minidom
from collections import Counter
from abnum import Abnum, greek

# greek abnum object for preprocess and isopsephical value getter
g = Abnum(greek)

# file to collect all stripped greek text
all_greek_text_file = "all_greek_text_files.txt"

# remove tags from the content
def remove_tags(x, corpus):
    x = re.sub('<[^<]+?>', '', x)
    if corpus == "greek_text_perseus":
        # with perseus data decode betacode to unicode
        x = betacode_to_unicode(x)
    return x

# remove all sub tags and their content. usually they are footnotes or other metadata
def soupit(txt, tag):
    soup = BeautifulSoup(txt, "lxml")
    root = getattr(soup, tag)
    return " ".join(filter(lambda x: len(x), list(c.strip() for c in root.children if "<" not in str(c))))

# parse greek text line for corpora
def line_for_corpora(node, tag, corpus):
    try:
        line = node.toxml().strip()
        if line:
            line = soupit(line, tag)
            if corpus == "greek_text_perseus":
                line = betacode_to_unicode(line)
            line = " ".join(filter(lambda x: x.strip() != "", line.split()))
            if line:
                return line.strip()
    except Exception as e:
        print(e)
        #print("Could not parse xml: %s %s" % (tag, corpus))
        pass
    return ''

# tags used to retrieve lines from xml file
tags = {
    "Aeschines": ["div1", ["head", "p"]],
    "Aeschylus": ["div1", ["speaker", "l"]],
    "Andocides": ["div1", ["head", "p"]],
    "Anth": ["div1", ["label", "p"]],
    "Apollodorus": ["div1", ["p"]],
    "Apollonius": ["div1", ["l"]],
    "Appian": ["p", []],
    "Aratus": ["p", []],
    "Aretaeus": ["p", ["foreign"]],
    "Aristides": ["body", ["head", "p"]],
    "Aristophanes": ["div1", ["speaker", "l"]],
    "Aristotle": ["p", []],
    "Arrian": ["body", ["p"]],
    "Athenaeus": ["body", ["p"]],
    "Bacchylides": ["div2", ["persName", "l"]],
    "Bible": ["div1", ["head", "p"]],
    "Callimachus": ["div1", ["head", "p"]],
    "Colluthus": ["p", []],
    "Demades": ["p", []],
    "Demosthenes": ["p", []],
    "Diodorus": ["p", []],
    "Diogenes": ["div1", ["head", "p"]],
    "Dionysius": ["p", []],
    "Dinarchus": ["p", []],
    "DioChrys": ["p", []],
    "Epictetus": ["p", []],
    "Euclid": ["p", []],
    "Galen": ["p", []],
    "Herodotus": ["p", []],
    "Hippocrates": ["p", []],
    "Hyperides": ["p", []],
    "JebbOrators": ["foreign", []],
    "Josephus": ["p", []],
    "Lucian": ["p", []],
    "Lycophron": ["p", []],
    "Lycurgus": ["div1", ["head", "p"]],
    "Lysias": ["div1", ["head", "p"]],
    "Oppian": ["p", []],
    "Nonnos": ["p", []],
    "Plato": ["p", []],
    "Plutarch": ["p", []],
    "Polybius": ["p", []],
    "Sibyl": ["div", ["head", "p"]],
    "Tryphiodorus": ["p", []],
    "Xenophon": ["p", []],
    "Lycophron": ["p", []],
    "Theophrastus": ["p", []],
    # tlg
    "tlg0643\\tlg001": ["div", ["l"]],
    "tlg0068\\tlg001": ["div", ["head", "l"]],
    "tlg0527\\tlg035": ["div", ["l"]],
    "tlg0084\\tlg001": ["div", ["head", "l"]],
    "tlg0643\\tlg002": ["div", ["l"]],
    "tlg0643\\tlg001": ["div", ["p"]],
    "tlg0011\\tlg003": ["div", ["l"]],
    "tlg0031\\tlg002": ["div", ["w"]],
    "tlg0527\\tlg029": ["div", ["l"]],
    "tlg0527\\tlg031": ["div", ["l"]],
    "tlg0527\\tlg032": ["div", ["l"]],
    "tlg0527\\tlg033": ["div", ["l"]],
    "tlg0527\\tlg034": ["div", ["l"]],
    "tlg0527\\tlg035": ["div", ["head", "l"]],
    "tlg0085\\tlg001": ["div", ["head", "speaker", "l"]],
    "tlg0011\\tlg003": ["div", ["speaker", "l"]],
    "tlg0643\\tlg001": ["div", ["l"]],
    "tlg0593\\1st1K001": ["div", ["head", "seq"]],
    "tlg1766\\tlg001": ["div", ["head", "seq"]],
    "tlg1220\\tlg001": ["div", ["head", "l"]],
    "tlg0643\\tlg001": ["div", ["head", "l"]],
    "tlg0069\\tlg001": ["div", ["head", "l"]],
}

# filter only greek xml files
def is_greek_file(x):
    return ("greek_orators" in x or \
            "attic_orators" in x or \
            "grc1.xml" in x or \
            "grc2.xml" in x or \
            "grc3.xml" in x or \
            "_01.xml" in x or "_02.xml" in x or \
            "_03.xml" in x or "_gk.xml" in x) and ".json" not in x

In [13]:
def init_corpora(corporas, tags):
    greek_corpora = []
    for corpus in corporas:
        d = path.join(dirt, corpus)
        files = []
        for a in listdir(d):
            # get only dirs
            if "." not in a:
                # again get only dirs containing xml files
                for b in filter(lambda x: "." not in x and "json" not in x, listdir(path.join(d, a))):
                    # build file paths to greek source files
                    e = joinpaths(d, [a, b])
                    files.extend(map(lambda x: path.join(e, x), filter(lambda x: is_greek_file(x), listdir(e))))
        # prepare corpora data
        for file in files:
            # defaul xml parse path
            if corpus == "greek_text_perseus":
                tg = ["l", []]
            else:
                tg = ["div", ["head", "p"]]
            # overwrite with custom parse path if exists
            for key, tag in tags.items():
                if key in file:
                    tg = tag
                    break
            greek_corpora.append({"tag": tg, "corpus": corpus, "uwords": {}, "length": 0,
                                  "file": file, "content": [], "simplified": ""})
    return greek_corpora

greek_corpora_x = init_corpora(corporas, tags)

In [14]:
len(greek_corpora_x)

1272

In [15]:
try:
    remove(all_greek_text_file)
except OSError:
    pass

mixed_content, mc = [], ["Ṃ", "Ḍ", "Ạ", "Ϙ", "Ê", "Ô", "Ϡ", "Ϟ", "Ϛ"]
mixed_content.extend(list(map(lambda x: x.lower(), mc)))

def has_mixed_content(data):
    a = []
    for x in mixed_content:
        if x in data:
            a.append(x)
    return a

roman_letters = "ABCDEFGHIJKLMNOPQRSTUVWXYZ";

def filter_only_greek_words(x):
    return len(x) and not any(map(lambda y: y in roman_letters, x))

def get_title_and_author(xmldoc, corpus, i):
    # parse title and author for storing temp files
    desc = xmldoc.getElementsByTagName('fileDesc')[0]
    title = desc.getElementsByTagName('title')[0].toxml().strip().replace("\n", " ")
    title = re.sub('<[^<]+?>', '', title)
    title = title.replace("?", "")
    title = title.replace("[", "")
    title = title.replace("]", "")
    title = title.strip(".").strip(" ")
    try:
        author = desc.getElementsByTagName('author')[0].toxml().strip().replace("\n", " ")
        author = re.sub('<[^<]+?>', '', author)
        author = author.replace("&gt;", "").replace(">", "").strip(" ")
    except Exception as e:
        author = str(i)
        i += 1
    if author == "":
        author = str(i)
        i += 1
    return g.preprocess(author).strip(), g.preprocess(title).strip(), i

def process_greek_corpora(greek_corpora):
    r, i = [], 0
    for corpus in greek_corpora:
        
        corp = corpus["corpus"]
        f = corpus["file"]

        try:
            xmldoc = minidom.parse(f)
        except Exception as e:
            print(e)
            continue

        author, title, i = get_title_and_author(xmldoc, corp, i)
        direct = path.join(corp, author)
        # create author based directory
        if not path.exists(direct):
            makedirs(direct)
        # init author and work based file paths
        f1 = path.join(direct, title + ".txt")
        f2 = path.join(direct, "Simplified " + title + ".txt")

        try:
            with open(f2, 'r', encoding="utf-8") as _:
                corpus['simplified'] = _.read()
        except:
            print("process file: " + f)
            if corpus['tag'][0] == "foreign":
                for item in xmldoc.getElementsByTagName("foreign"):
                    if item.hasAttribute("lang") and item.getAttribute("lang") == "greek":
                        corpus['content'].append(line_for_corpora(item, "foreign", corp))
            else:
                itemlist = xmldoc.getElementsByTagName(corpus['tag'][0])
                for item in itemlist:
                    if len(corpus['tag'][1]):
                        for tag in corpus['tag'][1]:
                            for item2 in item.getElementsByTagName(tag):
                                corpus['content'].append(line_for_corpora(item2, tag, corp))
                    else:
                        corpus['content'].append(line_for_corpora(item, corpus['tag'][0], corp))

            content = ' '.join(corpus['content'])
            corpus['simplified'] = ' '.join(filter(filter_only_greek_words, g.preprocess(content).upper().split()))
            if not corpus['simplified']:
                print("no data", corp, corpus['file'])
                continue
            else:
                append_to_file(f1, content, 'w')
                append_to_file(f2, corpus['simplified'], 'w')
        #
        append_to_file(all_greek_text_file, corpus['simplified'] + "\n")
        corpus['uwords'] = Counter(corpus['simplified'].split())
        corpus['length'] = len(corpus['simplified'].replace(" ", ""))
        
        print(corpus['file'], ":", title, ":", author, corpus['length'])
        
        if corpus['length'] < 100:
            print("min length", corp, corpus['file'], corpus['length'])
        else:
            corpus['lperw'] = corpus['length'] / len(corpus['uwords'])
        
        del corpus['content']
        r.append(corpus)
    return r

# collect greek corpora data
greek_corpora = process_greek_corpora(greek_corpora_x)

C:\Users\phtep\cltk_data\greek\text\greek_text_perseus\Aeschines\opensource\aeschin_gk.xml : Speeches Greek Machine readable text : Aeschines 218439
C:\Users\phtep\cltk_data\greek\text\greek_text_perseus\Aeschylus\opensource\aesch.ag_gk.xml : Agamemnon : Aeschylus 44201
C:\Users\phtep\cltk_data\greek\text\greek_text_perseus\Aeschylus\opensource\aesch.eum_gk.xml : Eumenides : Aeschylus 27990
C:\Users\phtep\cltk_data\greek\text\greek_text_perseus\Aeschylus\opensource\aesch.lib_gk.xml : Libation Bearers : Aeschylus 29135
C:\Users\phtep\cltk_data\greek\text\greek_text_perseus\Aeschylus\opensource\aesch.pb_gk.xml : Prometheus Bound : Aeschylus 31658
C:\Users\phtep\cltk_data\greek\text\greek_text_perseus\Aeschylus\opensource\aesch.pers_gk.xml : Persians : Aeschylus 28253
C:\Users\phtep\cltk_data\greek\text\greek_text_perseus\Aeschylus\opensource\aesch.seven_gk.xml : Seen Against Thebes : Aeschylus 28175
C:\Users\phtep\cltk_data\greek\text\greek_text_perseus\Aeschylus\opensource\aesch.supp_gk

C:\Users\phtep\cltk_data\greek\text\greek_text_perseus\Bible\opensource\nt_gk.xml : New Testament :  634592
C:\Users\phtep\cltk_data\greek\text\greek_text_perseus\Callimachus\opensource\call_02.xml : Hymns : Callimachus 30067
C:\Users\phtep\cltk_data\greek\text\greek_text_perseus\Callimachus\opensource\call_03.xml : Epigrams Fragmenta : Callimachus 9040
C:\Users\phtep\cltk_data\greek\text\greek_text_perseus\Colluthus\opensource\colluthus.01_gk.xml : Rape of Helen : Colluthus 14017
C:\Users\phtep\cltk_data\greek\text\greek_text_perseus\Demades\opensource\demad_gk.xml : On the Twele Years Greek Machine readable text : Demades 10439
C:\Users\phtep\cltk_data\greek\text\greek_text_perseus\Demosthenes\opensource\dem01-10_gk.xml : Speeches  Greek Machine readable text : Demosthenes 144210
C:\Users\phtep\cltk_data\greek\text\greek_text_perseus\Demosthenes\opensource\dem11-20_gk.xml : Speeches  Greek Machine readable text : Demosthenes 144210
C:\Users\phtep\cltk_data\greek\text\greek_text_perse

C:\Users\phtep\cltk_data\greek\text\greek_text_perseus\Homer\opensource\hom.il_gk.xml : Iliad Greek Machine readable text : Homer 551900
C:\Users\phtep\cltk_data\greek\text\greek_text_perseus\Homer\opensource\hom.od_gk.xml : Odyssey Greek Machine readable text : Homer 427044
C:\Users\phtep\cltk_data\greek\text\greek_text_perseus\Homeric_Hymns\opensource\hh_gk.xml : Homeric Hymns : Anonymous 82812
C:\Users\phtep\cltk_data\greek\text\greek_text_perseus\Hyperides\opensource\hyp_gk.xml : Speeches Greek Machine readable text : Hyperides 54001
C:\Users\phtep\cltk_data\greek\text\greek_text_perseus\JebbOrators\opensource\attic_orators.xml : The Attic Orators from Antiphon to Isaeos : Sir Richard C Jebb 59880
C:\Users\phtep\cltk_data\greek\text\greek_text_perseus\Josephus\opensource\j.aj_gk.xml : Antiquitates Judaicae : Flaius Josephus 1650744
C:\Users\phtep\cltk_data\greek\text\greek_text_perseus\Josephus\opensource\j.ap_gk.xml : Contra Apionem : Flaius Josephus 111855
C:\Users\phtep\cltk_dat

C:\Users\phtep\cltk_data\greek\text\greek_text_perseus\Lucian\opensource\63_gk.xml : Hermotimus : Lucian 67894
C:\Users\phtep\cltk_data\greek\text\greek_text_perseus\Lucian\opensource\64_gk.xml : Prometheus es in erbis : Lucian 5034
C:\Users\phtep\cltk_data\greek\text\greek_text_perseus\Lucian\opensource\65_gk.xml : Naigium : Lucian 26874
C:\Users\phtep\cltk_data\greek\text\greek_text_perseus\Lucian\opensource\66_gk.xml : Dialogi mortuorum : Lucian 54301
C:\Users\phtep\cltk_data\greek\text\greek_text_perseus\Lucian\opensource\67_gk.xml : Dialogi Marini : Lucian 19021
C:\Users\phtep\cltk_data\greek\text\greek_text_perseus\Lucian\opensource\68_gk.xml : Dialogi deorum : Lucian 35841
C:\Users\phtep\cltk_data\greek\text\greek_text_perseus\Lucian\opensource\69_gk.xml : Dialogi meretricii : Lucian 38535
C:\Users\phtep\cltk_data\greek\text\greek_text_perseus\Lucian\opensource\70_gk.xml : Soleocista : Lucian 8110
C:\Users\phtep\cltk_data\greek\text\greek_text_perseus\Lucian\opensource\71_gk.xml

C:\Users\phtep\cltk_data\greek\text\greek_text_perseus\Plutarch\opensource\plut.086_teubner_gk.xml : De fortuna Romanorum : Plutarch 23801
C:\Users\phtep\cltk_data\greek\text\greek_text_perseus\Plutarch\opensource\plut.087_loeb_gk.xml : De Alexandri magni fortuna aut irtute : Plutarch 38894
C:\Users\phtep\cltk_data\greek\text\greek_text_perseus\Plutarch\opensource\plut.087_teubner_gk.xml : De Alexandri magni fortuna aut irtute : Plutarch 38894
C:\Users\phtep\cltk_data\greek\text\greek_text_perseus\Plutarch\opensource\plut.088_loeb_gk.xml : De gloria Atheniensium : Plutarch 13983
C:\Users\phtep\cltk_data\greek\text\greek_text_perseus\Plutarch\opensource\plut.088_teubner_gk.xml : De gloria Atheniensium : Plutarch 13983
C:\Users\phtep\cltk_data\greek\text\greek_text_perseus\Plutarch\opensource\plut.089_teubner_gk.xml : De Iside et Osiride : Plutarch 81216
C:\Users\phtep\cltk_data\greek\text\greek_text_perseus\Plutarch\opensource\plut.090_teubner_gk.xml : De E apud Delphos : Plutarch 3274


...

C:\Users\phtep\cltk_data\greek\text\greek_text_tlg\tlg2200\tlg00551\tlg2200.tlg00551.opp-grc1.xml : Declamatio : Libanius 25548
process file: C:\Users\phtep\cltk_data\greek\text\greek_text_tlg\tlg2200\tlg008\tlg2200.tlg008.opp-grc1.xml
C:\Users\phtep\cltk_data\greek\text\greek_text_tlg\tlg2200\tlg008\tlg2200.tlg008.opp-grc1.xml : Characteres Epistolici : Libanius 23632
process file: C:\Users\phtep\cltk_data\greek\text\greek_text_tlg\tlg2233\tlg001\tlg2233.tlg001.1st1K-grc1.xml
C:\Users\phtep\cltk_data\greek\text\greek_text_tlg\tlg2233\tlg001\tlg2233.tlg001.1st1K-grc1.xml : Physiognomonica Epitome Adamantiana : PseudoPolemon 112995
process file: C:\Users\phtep\cltk_data\greek\text\greek_text_tlg\tlg2233\tlg002\tlg2233.tlg002.1st1K-grc1.xml
C:\Users\phtep\cltk_data\greek\text\greek_text_tlg\tlg2233\tlg002\tlg2233.tlg002.1st1K-grc1.xml : Physiognomonica : PseudoPolemon 5544
process file: C:\Users\phtep\cltk_data\greek\text\greek_text_tlg\tlg2371\tlg001\tlg2371.tlg001.opp-grc1.xml
C:\Users

process file: C:\Users\phtep\cltk_data\greek\text\greek_text_tlg\tlg4013\tlg005\tlg4013.tlg005.opp-grc1.xml
C:\Users\phtep\cltk_data\greek\text\greek_text_tlg\tlg4013\tlg005\tlg4013.tlg005.opp-grc1.xml : In Aristotelis libros de anima commentaria Sp fort auctore Prisciano Lydo : Simplicius 1192020
process file: C:\Users\phtep\cltk_data\greek\text\greek_text_tlg\tlg4015\tlg001\tlg4015.tlg001.opp-grc1.xml
C:\Users\phtep\cltk_data\greek\text\greek_text_tlg\tlg4015\tlg001\tlg4015.tlg001.opp-grc1.xml : In Aristotelis Categorias Commentarium : Joannes Philoponus 579976
process file: C:\Users\phtep\cltk_data\greek\text\greek_text_tlg\tlg4015\tlg002\tlg4015.tlg002.opp-grc1.xml
C:\Users\phtep\cltk_data\greek\text\greek_text_tlg\tlg4015\tlg002\tlg4015.tlg002.opp-grc1.xml : in Aristotelis Analytica Priora commentaria : Joannes Philoponus 1414828
process file: C:\Users\phtep\cltk_data\greek\text\greek_text_tlg\tlg4015\tlg004\tlg4015.tlg004.opp-grc1.xml
C:\Users\phtep\cltk_data\greek\text\greek_tex

C:\Users\phtep\cltk_data\greek\text\greek_text_tlg\tlg4036\tlg001\tlg4036.tlg001.opp-grc1.xml : In Platonis Rem Publicam Commentarii : Proclus Diadochus 1487168
process file: C:\Users\phtep\cltk_data\greek\text\greek_text_tlg\tlg4072\tlg001\tlg4072.tlg001.1st1K-grc1.xml
C:\Users\phtep\cltk_data\greek\text\greek_text_tlg\tlg4072\tlg001\tlg4072.tlg001.1st1K-grc1.xml : Commentarii in libros de sphaera et cylindro : Eutocius 223054
process file: C:\Users\phtep\cltk_data\greek\text\greek_text_tlg\tlg4072\tlg002\tlg4072.tlg002.1st1K-grc1.xml
C:\Users\phtep\cltk_data\greek\text\greek_text_tlg\tlg4072\tlg002\tlg4072.tlg002.1st1K-grc1.xml : Commentarius in dimensionem circuli : Eutocius 43542
process file: C:\Users\phtep\cltk_data\greek\text\greek_text_tlg\tlg4072\tlg003\tlg4072.tlg003.1st1K-grc1.xml
C:\Users\phtep\cltk_data\greek\text\greek_text_tlg\tlg4072\tlg003\tlg4072.tlg003.1st1K-grc1.xml : Commentarius in libros de planorum aequilibriis : Eutocius 81120
process file: C:\Users\phtep\cltk_

process file: C:\Users\phtep\cltk_data\greek\text\greek_text_tlg\tlg4102\tlg038\tlg4102.tlg038.opp-grc1.xml
C:\Users\phtep\cltk_data\greek\text\greek_text_tlg\tlg4102\tlg038\tlg4102.tlg038.opp-grc1.xml : Catena In Epistulam Ad Hebraeos : Catenae Noum Testamentum 1434390
process file: C:\Users\phtep\cltk_data\greek\text\greek_text_tlg\tlg4102\tlg040\tlg4102.tlg040.opp-grc1.xml
C:\Users\phtep\cltk_data\greek\text\greek_text_tlg\tlg4102\tlg040\tlg4102.tlg040.opp-grc1.xml : Catena In Epistulam Jacobi : Catenae Noum Testamentum 103896
process file: C:\Users\phtep\cltk_data\greek\text\greek_text_tlg\tlg4102\tlg041\tlg4102.tlg041.opp-grc1.xml
C:\Users\phtep\cltk_data\greek\text\greek_text_tlg\tlg4102\tlg041\tlg4102.tlg041.opp-grc1.xml : Catena In Epistulam Petri I : Catenae Noum Testamentum 111806
process file: C:\Users\phtep\cltk_data\greek\text\greek_text_tlg\tlg4102\tlg042\tlg4102.tlg042.opp-grc1.xml
C:\Users\phtep\cltk_data\greek\text\greek_text_tlg\tlg4102\tlg042\tlg4102.tlg042.opp-grc1.

C:\Users\phtep\cltk_data\greek\text\greek_text_tlg\tlg5034\tlg001c\tlg5034.tlg001c.perseus-grc1.xml : Scholia in Pindarum Nemean Odes : Pindar Scholia 508823
process file: C:\Users\phtep\cltk_data\greek\text\greek_text_tlg\tlg5034\tlg001d\tlg5034.tlg001d.perseus-grc1.xml
C:\Users\phtep\cltk_data\greek\text\greek_text_tlg\tlg5034\tlg001d\tlg5034.tlg001d.perseus-grc1.xml : Scholia in Pindarum Isthmian Odes : Pindar Scholia 239929
process file: C:\Users\phtep\cltk_data\greek\text\greek_text_tlg\tlg9004\tlg001\tlg9004.tlg001.opp-grc1.xml
C:\Users\phtep\cltk_data\greek\text\greek_text_tlg\tlg9004\tlg001\tlg9004.tlg001.opp-grc1.xml : Anonymi in analyticorum posteriorum librum alterum commentarium : Anonymi In Aristotelis Librum Alterum Analyticorum Posteriorum 141860
process file: C:\Users\phtep\cltk_data\greek\text\greek_text_tlg\tlg9006\tlg011\tlg9006.tlg011.opp-grc1.xml
C:\Users\phtep\cltk_data\greek\text\greek_text_tlg\tlg9006\tlg011\tlg9006.tlg011.opp-grc1.xml : Libanius Opera : Libaniu

## Statistics

Get the size of the all concatenated texts:

In [16]:
# python 3.4+ 
# https://stackoverflow.com/questions/2104080/how-to-check-file-size-in-python
from pathlib import Path
file = Path() / all_greek_text_file
size = file.stat().st_size
print("Size of the raw text: %s MB" % round(size/1024/1024, 2))

Size of the raw text: 606.04 MB


Calculate the other statistics of the saved text file:

In [17]:
# read file
with open(all_greek_text_file, 'r', encoding="utf-8") as f:
    content = f.read().replace("\n", " ")

# remove space chars to get pure raw greek character string of the file
ccontent = content.replace(" ", "")
chars = len(ccontent)
# split by space char to get words
words = content.split()
lwords = len(words)
# unique words
uwords = set(words)
luwords = len(uwords)
# output info
print("Letters: " + str(chars))
print("Words in total: " + str(lwords))
print("Unique words: " + str(luwords))

Letters: 288197493
Words in total: 59086456
Unique words: 825088


## Letter statistics

Use Pandas library to handle tabular data and show basic character statistics. The first column is the character, the second column is the count of the letter, and the third column is the percentage of the letter contra all letters.

In [18]:
from pandas import DataFrame
df = DataFrame([[k, v] for k, v in Counter(ccontent).items()])
df[2] = df[1].apply(lambda x: round(x*100/chars, 2))

In [20]:
df[df[1] > 20].sort_values(1)

,0,1,2
27,Ϟ,204,0.00
28,Ϡ,364,0.00
25,ϒ,8225,0.00
26,Ϛ,8430,0.00
17,Ψ,421972,0.15
24,Ϲ,555696,0.19
19,Ζ,629980,0.22
23,Ξ,1094906,0.38
22,Β,1532950,0.53
16,Φ,2427462,0.84


In [134]:
# read unique words stats if available
csv_file_name = "greek_words_corpora.csv"
try:
    from pandas import read_csv
    df = read_csv(csv_file_name, header = None)
    df[2] = df[2].apply(lambda x: float(x))
    df[3] = df[3].apply(lambda x: int(x))
    df[4] = df[4].apply(lambda x: int(x))
    df[5] = df[5].apply(lambda x: str(x).replace("'", "").replace("[", "").replace("]", "").split(", "))
    df[6] = df[6].apply(lambda x: int(x))
except:
    # lets count unique words statistic from the parsed greek corpora rather than the plain text file
    # it would be pretty dauntful to find out occurence of the all 800000+ unique words from the text 
    # file that is over 600 MB big!
    unique_word_stats = {}
    for item in greek_corpora:
        for word, cnt in item['uwords'].items():
            if word not in unique_word_stats:
                unique_word_stats[word] = 0
            unique_word_stats[word] += cnt
    # init dataframe
    df = DataFrame([[k, v] for k, v in unique_word_stats.items()])
    # add column for the occurrence percentage of the word
    df[2] = df[1].apply(lambda x: round(x*100/lwords, 2))
    # add column for the length of the word
    df[3] = df[0].apply(lambda x: len(x))
    # add isopsephy column
    df[4] = df[0].apply(lambda x: g.value(x))
    from greek_accentuation.syllabify import syllabify
    # add syllabified column
    df[5] = df[0].apply(lambda x: syllabify(x))
    # add length of the syllables column
    df[6] = df[5].apply(lambda x: len(x))
    df.to_csv(csv_file_name, header=False, index=False, encoding='utf-8')

In [135]:
# most repeated words
df.sort_values(1, ascending=False)

,0,1,2,3,4,5,6
38,ΚΑΙ,3330981,5.64,3,31,[ΚΑΙ],1
25,ΔΕ,1361502,2.30,2,9,[ΔΕ],1
309,ΤΟ,1291657,2.19,2,370,[ΤΟ],1
46,ΤΟΥ,927927,1.57,3,770,[ΤΟΥ],1
2,ΤΩΝ,914800,1.55,3,1150,[ΤΩΝ],1
101,Η,880934,1.49,1,8,[Η],1
26,ΤΗΝ,800865,1.36,3,358,[ΤΗΝ],1
11,ΕΝ,710521,1.20,2,55,[ΕΝ],1
224,Ο,673358,1.14,1,70,[Ο],1
216,ΤΗΣ,671515,1.14,3,508,[ΤΗΣ],1


In [110]:
# longest words
#df.sort_values(3, ascending=False)

In [111]:
# least repeated words and how many percent of the whole word base
for x, y in df.groupby([1, 2]).count()[:10].T.items():
    print("words repeating %s time(s): " % x[0], round(100*y[0]/825088, 2), "%")

words repeating 1 time(s):  13.39 %
words repeating 2 time(s):  14.41 %
words repeating 3 time(s):  17.42 %
words repeating 4 time(s):  10.73 %
words repeating 5 time(s):  3.59 %
words repeating 6 time(s):  5.37 %
words repeating 7 time(s):  2.45 %
words repeating 8 time(s):  3.41 %
words repeating 9 time(s):  2.2 %
words repeating 10 time(s):  1.71 %


In [112]:
# compare corporas and their stats

In [113]:
# longest words reveals that some of the texts are not separated by spaces in the original text
# so we lack of the definite meaning for these. also they could be removed from stats by occurrence is 1
# and length is above 30 chars of something...
#df.sort_values(3, ascending=False)

## Solve the riddle

Using parameters from the riddle, lets try to solve it by brute force.

- knowing isopsephical value 1697
- knowing the length of the word
- knowing the details of the syllables of the word
- comparing the context of the result

How many words there are with the isopsephical value 1697 and letters counting nine?

In [119]:
# retrieve the words having nine letters and their value as 1697
#df[df[4] == 1697][df[0].str.len() == 9]

It turns out that there are just very few words meeting the criteria of the riddle. We could already make analyzing of the words manually.

The last interest is to split words to syllables so that we can make the final conclusion. I will however add one extension to the search and allow the count of the letters to be between 8 and 10. That is due to double consonant and long / short vowelspecialty of the Greek language.

In [121]:
# we might want to make another filter by using syllabification for the word and count
# only those words having the length between 8 and 10 (short and long vowels included)
#df[df[6] == 4]

In [123]:
# more strictly only words with 9 letters and 4 syllables
#df[df[6] == 4][df[3] == 9]

In [124]:
# words having 9 letters, 4 syllables where the first three of the syllables has two letters, last one the rest 3
#d = df[df[6] == 4][df[3] == 9]
#d[d.apply(lambda x: len(x[5][0]) == 2 and len(x[5][1]) == 2 and len(x[5][2]) == 2, axis=1)]

Finally there is the rule of five consonants in the word in the original riddle. Lets refactor all of this and the previous ones to the single callable function and apply it to dataframe.

In [137]:
# the word should have n mutes ie consonants
consonants = "ΨΖΞΒΦΧΘΓΔΜΛΚΠΡΣΤΝϹϚϠϞ"
#consonants = "ΨΖΞΒΦΧΘΓΔΜΛΚΠΡΣΤΝ"
def nmutes(x, n):
    tot = 0
    for c in consonants:
        tot += x[0].count(c)
        if tot > n:
            return False
    return tot == n

# the word should have n vowels
vowels = "ϒΩΗΥΕΙΟΑ"
def nvowels(x, n):
    tot = 0
    for c in vowels:
        tot += x[0].count(c)
        if tot > n:
            return False
    return tot == n

# the word should have n syllables
def nsyllables(x, n):
    return x[6] == n

# the word should have two letters in the first three syllables, and the rest (3) letters in the last
def has_two_letters_in_first_three_syllables(x):
    return len(x[5][0]) == 2 and len(x[5][1]) == 2 and len(x[5][2]) == 2

# this makes n letters in total
def nletters(x, n):
    return len(x[0]) == n

# isopsephical value
def nisopsephy(x, n):
    return x[4] > n[0] and x[4] < n[1] if type(n) is list else x[4] == n

# riddle wrapper function
def riddle(x, isopsephy, letters = 9, mutes = 5, syllables = 4):
    return nisopsephy(x, isopsephy) and \
           nsyllables(x, syllables) and \
           nletters(x, letters) and \
           nmutes(x, mutes) and \
           has_two_letters_in_first_three_syllables(x)

In [138]:
# solve the riddle
c = df[df.apply(lambda x: riddle(x, 1697), axis=1)]
c

,0,1,2,3,4,5,6
37510,ΑΜΦΕΚΑΛΥΨ,1,0.0,9,1697,"[ΑΜ, ΦΕ, ΚΑ, ΛΥΨ]",4
341688,ΣΥΝΩΚΙΣΘΗ,19,0.0,9,1697,"[ΣΥ, ΝΩ, ΚΙ, ΣΘΗ]",4
711650,ΛΗΛΥΘΟΤΩΝ,3,0.0,9,1697,"[ΛΗ, ΛΥ, ΘΟ, ΤΩΝ]",4


Thus we have found one good match for the riddle. The word is:

# ΑΜΦΕΚΑΛΥΨ

(amphekalyps / amfecalyps) meaning all covering.

Next we should make some text and linquistic examination, how well the proposed word fits to the immediate context of the sibylline verses. Where are the exact occurrences of the word in the Greek corpora, in which context? Is it a word suitable for an epithet, does it have any religious spiritual significance, and so forth. These questions I will leave for the other study.

In [40]:
for x, y in c.T.items():
    for item in greek_corpora:
        if y[0] in item['simplified'].split():
            print("Exact: ", y[0], item['file'].replace(dirt, ""))
        elif y[0] in item['simplified']:
            print("Partial: ", y[0], item['file'].replace(dirt, ""))

Exact:  ΑΜΦΕΚΑΛΥΨ \greek_text_perseus\Anth\opensource\02_gk.xml
Partial:  ΑΜΦΕΚΑΛΥΨ \greek_text_perseus\Anth\opensource\03_gk.xml
Partial:  ΑΜΦΕΚΑΛΥΨ \greek_text_perseus\Apollonius\opensource\argo_gk.xml
Partial:  ΑΜΦΕΚΑΛΥΨ \greek_text_perseus\Hesiod\opensource\hes.wd_gk.xml
Partial:  ΑΜΦΕΚΑΛΥΨ \greek_text_perseus\Homer\opensource\hom.il_gk.xml
Partial:  ΑΜΦΕΚΑΛΥΨ \greek_text_perseus\Homer\opensource\hom.od_gk.xml
Partial:  ΑΜΦΕΚΑΛΥΨ \greek_text_perseus\Nonnos\opensource\nonnos_01.xml
Partial:  ΑΜΦΕΚΑΛΥΨ \greek_text_perseus\Sibyl\opensource\tlg1551.tlg001.opp-grc1_gk.xml
Partial:  ΑΜΦΕΚΑΛΥΨ \greek_text_tlg\tlg1220\tlg001\tlg1220.tlg001.1st1K-grc1.xml
Partial:  ΑΜΦΕΚΑΛΥΨ \greek_text_tlg\tlg1551\tlg001\tlg1551.tlg001.opp-grc1.xml
Partial:  ΑΜΦΕΚΑΛΥΨ \greek_text_tlg\tlg5026\tlg001\tlg5026.tlg001.1st1K-grc1.xml
Partial:  ΑΜΦΕΚΑΛΥΨ \greek_text_tlg\tlg5026\tlg007\tlg5026.tlg007.First1K-grc1.xml
Partial:  ΣΥΝΩΚΙΣΘΗ \greek_text_perseus\Aristides\opensource\aristid.orat_gk.xml
Partial:  ΣΥΝΩΚΙΣ

## The [MIT](http://choosealicense.com/licenses/mit/) License

Copyright &copy; 2018 Marko Manninen